In [66]:
from pyspark.sql import SparkSession
import re
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Learning_Spark") \
    .getOrCreate()
sc = spark.sparkContext

In [67]:
#Function to parse the XML file and handle the Key Value pair efficiently
def xml2rdd(filename, line2, firstSplitter,  line1 = 'version="1.0" encoding="utf-8"'):
    users = sc.textFile(filename)    
    users = users.map(lambda x: x.replace("&quot;", '').replace("&#xD;", '').replace("&#xA;", '').replace("&lt;", '').replace("&gt;", '').replace("&amp;", ''))
    users = users.filter(lambda x: line1 not in x)
    users = users.filter(lambda x: line2 not in x)
    users = users.map(lambda x: (x.split(firstSplitter)[0].strip().replace('<row Id="', '')[:-1], (x.replace('/>','').replace('" ', '"|').split("|")[1:-1])))
    users = users.mapValues(lambda x: [tuple(item.replace('="', '|~|"').split('|~|')) for item in x])
    #users = users.mapValues(lambda x: dict([tuple(item.replace('="', '|~|"').split('|~|')) for item in x]))
    return(users)

In [68]:
users = xml2rdd("Users.xml", '<users>', 'Reputation=')
users.take(12)

[('-1',
  [('Reputation', '"1"'),
   ('CreationDate', '"2014-05-13T21:29:22.820"'),
   ('DisplayName', '"Community"'),
   ('LastAccessDate', '"2014-05-13T21:29:22.820"'),
   ('WebsiteUrl', '"http://meta.stackexchange.com/"'),
   ('Location', '"on the server farm"'),
   ('AboutMe',
    '"pHi, I\'m not really a person./ppI\'m a background process that helps keep this site clean!/ppI do things like/pulliRandomly poke old unanswered questions every hour so they get some attention/liliOwn community questions and answers so nobody gets unnecessary reputation from them/liliOwn downvotes on spam/evil posts that get permanently deleted/liliOwn suggested edits from anonymous users/lilia href=http://meta.stackexchange.com/a/92006Remove abandoned questions/a/li/ul"'),
   ('Views', '"0"'),
   ('UpVotes', '"798"'),
   ('DownVotes', '"1325"'),
   ('AccountId', '"-1"')]),
 ('1',
  [('Reputation', '"101"'),
   ('CreationDate', '"2014-05-13T22:58:54.810"'),
   ('DisplayName', '"Adam Lear"'),
   ('LastAc

In [69]:
comments = xml2rdd('Comments.xml', '<comments>', 'PostId=')
comments.take(10)

[('5',
  [('PostId', '"5"'),
   ('Score', '"9"'),
   ('Text',
    '"this is a super theoretical AI question. An interesting discussion! but out of place..."'),
   ('CreationDate', '"2014-05-14T00:23:15.437"'),
   ('UserId', '"34"')]),
 ('6',
  [('PostId', '"7"'),
   ('Score', '"4"'),
   ('Text',
    '"List questions are usually not suited for Stack Exchange websites since there isn\'t an objective answer or a way to measure the usefulness of an answer. Having said that, one of my recommendations would be MacKay\'s Information Theory, Inference, and Learning Algorithms."'),
   ('CreationDate', '"2014-05-14T00:38:19.510"'),
   ('UserId', '"51"')]),
 ('9',
  [('PostId', '"7"'),
   ('Score', '"3"'),
   ('Text',
    '"This question appears to be off-topic because it is asks for a favorite resource.  On other SE sites, this would immediately be closed.  Since this is a new site, we still have to decide if this is a valid question here"'),
   ('CreationDate', '"2014-05-14T01:16:12.623"'),
   

In [70]:
ph = xml2rdd('PostHistory.xml', '<posthistory>', 'PostHistoryTypeId=')
ph.take(5)

[('7',
  [('PostHistoryTypeId', '"2"'),
   ('PostId', '"5"'),
   ('RevisionGUID', '"009bca93-fce2-44ed-a277-a8452650a627"'),
   ('CreationDate', '"2014-05-13T23:58:30.457"'),
   ('UserId', '"5"'),
   ('Text',
    '"I\'ve always been interested in machine learning, but I can\'t figure out one thing about starting out with a simple Hello World example - how can I avoid hard-coding behavior?For example, if I wanted to teach a bot how to avoid randomly placed obstacles, I couldn\'t just use relative motion, because the obstacles move around, but I don\'t want to hard code, say, distance, because that ruins the whole point of machine learning.Obviously, randomly generating code would be impractical, so how could I do this?"')]),
 ('8',
  [('PostHistoryTypeId', '"1"'),
   ('PostId', '"5"'),
   ('RevisionGUID', '"009bca93-fce2-44ed-a277-a8452650a627"'),
   ('CreationDate', '"2014-05-13T23:58:30.457"'),
   ('UserId', '"5"'),
   ('Text',
    '"How can I do simple machine learning without hard-c

### From the Users.xml file, find all users which are from Georgia and output to screen their DisplayName only.

In [71]:
a = users.mapValues(lambda x: [item for item in x if (item[0] == 'Location' and 'GA' in item[1])] + [item[1] for item in x if 'DisplayName' in item[0]])
a = a.values()
a = a.filter(lambda x: len(x) > 1)
a = a.map(lambda x: x[1].replace('"',''))
print("Answer for first question:\nAll users which are from Georgia:\n")
a.collect()

Answer for first question:
All users which are from Georgia:



['Tony Boyles',
 'pkerl',
 'Nick Larsen',
 'gfritz',
 'Aleksandr Blekh',
 'Michael',
 'azoorob',
 'ontek',
 'Aravind R. Yarram',
 'ilya',
 'Daisuke Aramaki',
 'tempusfugit',
 'Henry Crutcher',
 'Goddard',
 'Matt Simpson',
 'Peter Woolfitt',
 'matt biskup',
 'Jason W',
 'Peter Mourfield',
 'Magsol',
 'Bob Baxley',
 'badjr',
 'mplunney',
 'YC Hu',
 'ryan',
 'Patrick Gerbes',
 'Ilya Lapitan',
 'pradyumnad',
 'Psidom',
 'Teresa Madsen',
 'Brandon',
 'jpm',
 'Mr. Rooter of Savannah',
 'Mr. Rooter of Southeast GA',
 'Khiem Ha',
 'Ahmet Cecen',
 'Guy Gordon',
 'C3Theo',
 'niru dyogi',
 'Vinitha Palani',
 'Mac18',
 'Andrew',
 'Aditya Gogoi',
 'turtlemonvh',
 'Lewis Rodgers',
 'Tarun Luthra',
 'Devendra Lattu',
 'cosmosa',
 'Todd Dawson',
 'ironv',
 'Mboolean',
 'Jimd',
 'David F',
 'PSInf',
 'Chirag',
 'Sandeep Gunda',
 'hellofanengineer',
 'Will Gao',
 'Oriol Mirosa',
 'Andrew King',
 'David',
 'rajb245',
 'Sealander',
 'afshin',
 'Ashish Powani',
 'Boris N.',
 'Atul Kaushik',
 'Harnoor Singh

### Using the Users.xml file, provide the count for all users which joined (CreationDate) in 2017

In [72]:
a = users.mapValues(lambda x: [item for item in x if (item[0] == 'CreationDate' and '2017-' in item[1])] )
a = a.values()
a = a.filter(lambda x: len(x) > 0)
#a = a.map(lambda x: x[1].replace('"',''))
print("Answer for second question:\nCount for all users which joined (CreationDate) in 2017= ", a.count())

Answer for second question:
Count for all users which joined (CreationDate) in 2017=  14239


### Using the PostHistory file, count the number of Posts that feature the words “Spark” and “Scala”. Output this to the screen.

In [73]:
a = ph.map(lambda x: [item for item in x[1] if(item[0] == 'Text' and 'Spark' in item[1] and 'Scala' in item[1])])
a = a.filter(lambda x: len(x) > 0)
print("Answer for third question:\nCount for Posts that feature the words “Spark” and “Scala” = ", a.count())

Answer for third question:
Count for Posts that feature the words “Spark” and “Scala” =  94


### Using the PostHistory file, provide a total count of the words used by each distinct user. In other words, count all words in all posts for each user and display this to screen. You can only identify users by the UserID (30 points). You get 15 bonus points if you get the actual DisplayName of the user.

In [74]:
u = users.mapValues(lambda x: ' '.join([item[1].replace('"', '') for item in x if item[0] == 'DisplayName']) )
u.collect()

[('-1', 'Community'),
 ('1', 'Adam Lear'),
 ('2', 'Geoff Dalgas'),
 ('3', 'hichris123'),
 ('4', 'Ben Collins'),
 ('5', 'Doorknob'),
 ('6', 'gerrit'),
 ('7', 'Undo'),
 ('8', 'Jon Ericson'),
 ('9', 'rolfl'),
 ('10', 'Dave Kincaid'),
 ('11', 'Yavar'),
 ('12', 'hairboat'),
 ('14', 'Tim Goodman'),
 ('15', 'Tony Boyles'),
 ('16', 'Jeremy Banks'),
 ('17', 'Matias Valdenegro'),
 ('18', 'Gilles'),
 ('19', 'Max Ionov'),
 ('20', 'TheDoctor'),
 ('21', 'Sean Owen'),
 ('22', 'Nick Peterson'),
 ('23', 'zengr'),
 ('24', 'buruzaemon'),
 ('25', 'kabb'),
 ('26', 'Alex I'),
 ('27', 'Cole'),
 ('28', 'JorgeArtware'),
 ('29', 'Eric Wilson'),
 ('30', 'Sklivvz'),
 ('31', 'Olexandr Isayev'),
 ('32', 'Adam Barthelson'),
 ('33', 'Uri Laserson'),
 ('34', 'Vass'),
 ('35', 'Jaydles'),
 ('36', 'statsRus'),
 ('37', 'Revious'),
 ('38', 'Josh Crozier'),
 ('39', 'user1754606'),
 ('40', 'rateldajer'),
 ('41', 'Ana'),
 ('42', 'Max Makarochkin'),
 ('43', 'cynddl'),
 ('45', 'user88377'),
 ('46', 'WOPR'),
 ('47', 'ailnlv'),
 

In [75]:
a = ph.map(lambda x: [item for item in x[1] if(item[0] in ['UserId', 'Text'])])
a = a.filter(lambda x: len(x) > 1)
a = a.map(lambda x: (x[0][1].replace('"', ''), x[1][1]))
a = a.reduceByKey(lambda x, y: x + y)
a = a.mapValues(lambda x: [re.sub(r"\s*[^A-Za-z]+\s*", " ", item).strip() for item in x.replace('""', ' ').split(' ')])
a = a.mapValues(lambda x: str(len([item for item in x if len(item) > 0])) + ' Words')
a.collect()

[('22', '173 Words'),
 ('66', '216 Words'),
 ('63', '297 Words'),
 ('97', '6145 Words'),
 ('118', '701 Words'),
 ('53', '688 Words'),
 ('132', '808 Words'),
 ('134', '1821 Words'),
 ('151', '235 Words'),
 ('119', '419 Words'),
 ('178', '1675 Words'),
 ('24', '1266 Words'),
 ('189', '317 Words'),
 ('199', '305 Words'),
 ('113', '638 Words'),
 ('34', '666 Words'),
 ('70', '270 Words'),
 ('212', '71 Words'),
 ('122', '3390 Words'),
 ('227', '1664 Words'),
 ('201', '24551 Words'),
 ('262', '346 Words'),
 ('275', '228 Words'),
 ('278', '82 Words'),
 ('71', '367 Words'),
 ('295', '450 Words'),
 ('43', '123 Words'),
 ('432', '250 Words'),
 ('451', '414 Words'),
 ('456', '204 Words'),
 ('454', '1109 Words'),
 ('498', '140 Words'),
 ('514', '1113 Words'),
 ('531', '133 Words'),
 ('655', '70 Words'),
 ('680', '45 Words'),
 ('471', '8406 Words'),
 ('714', '88 Words'),
 ('839', '140 Words'),
 ('515', '191 Words'),
 ('872', '265 Words'),
 ('889', '140 Words'),
 ('322', '8108 Words'),
 ('901', '207 

In [76]:
a = u.leftOuterJoin(a)
a = a.values()
a = a.map(lambda x: str(x[0]) + ' - ' + str(x[1]).replace('None', '0 Words'))
print("Answer for fourth question:\nCount  of the words used by each distinct user:\n ")
a.collect()

Answer for fourth question:
Count  of the words used by each distinct user:
 


['Doorknob - 95 Words',
 'Undo - 0 Words',
 'Dave Kincaid - 0 Words',
 'Josh Crozier - 0 Words',
 'user1754606 - 0 Words',
 'Max Makarochkin - 0 Words',
 'user88377 - 0 Words',
 'ailnlv - 0 Words',
 'Robert Cartaino - 6 Words',
 'leventov - 0 Words',
 'NivF007 - 0 Words',
 'Shog9 - 403 Words',
 'Stuart - 0 Words',
 'Rob Hoare - 0 Words',
 'Nour - 0 Words',
 'ThisIzKp - 0 Words',
 'user3554712 - 0 Words',
 'Nick Craver - 0 Words',
 'jonsca - 0 Words',
 'Igor Bobriakov - 351 Words',
 'prthkms - 0 Words',
 'EdChum - 1178 Words',
 'Johnny000 - 2197 Words',
 'jcataluna - 0 Words',
 'Konstantin V. Salikhov - 701 Words',
 'frandiaz83 - 0 Words',
 'Rhand - 182 Words',
 'Simon - 0 Words',
 'joker - 282 Words',
 'DVK - 808 Words',
 'akellyirl - 250 Words',
 'I11umin8 - 0 Words',
 'Marco13 - 2766 Words',
 'willwest - 0 Words',
 'Wojciech Walczak - 2983 Words',
 'samthebrand - 182 Words',
 'Marco Poli - 0 Words',
 'Learner - 0 Words',
 'Travis J - 0 Words',
 'GIRI - 0 Words',
 'Filipe Ferminiano -

### GRADUATE STUDENTS: Using the users.xml, comments.xml and PostHistory.xml files, produce a single file that includes the following information: DisplayName, Number of Comments, total Score and Number of posts. This file should have the users (DisplayName) sorted by score, descending from higher to lower. (40 points)

In [77]:
c = comments
u = users
p = ph
c.take(5)

[('5',
  [('PostId', '"5"'),
   ('Score', '"9"'),
   ('Text',
    '"this is a super theoretical AI question. An interesting discussion! but out of place..."'),
   ('CreationDate', '"2014-05-14T00:23:15.437"'),
   ('UserId', '"34"')]),
 ('6',
  [('PostId', '"7"'),
   ('Score', '"4"'),
   ('Text',
    '"List questions are usually not suited for Stack Exchange websites since there isn\'t an objective answer or a way to measure the usefulness of an answer. Having said that, one of my recommendations would be MacKay\'s Information Theory, Inference, and Learning Algorithms."'),
   ('CreationDate', '"2014-05-14T00:38:19.510"'),
   ('UserId', '"51"')]),
 ('9',
  [('PostId', '"7"'),
   ('Score', '"3"'),
   ('Text',
    '"This question appears to be off-topic because it is asks for a favorite resource.  On other SE sites, this would immediately be closed.  Since this is a new site, we still have to decide if this is a valid question here"'),
   ('CreationDate', '"2014-05-14T01:16:12.623"'),
   

In [78]:
#Calculating number of comments
ct = c.map(lambda x: [item for item in x[1] if item[0] in ['UserId', 'Text']])
ct = ct.filter(lambda x: len(x) > 1)
ct = ct.map(lambda x: (x[1][1].replace('"', ''), 1))
ct = ct.reduceByKey(lambda x,y : x + y)
ct = ct.mapValues(lambda x: (str(x) + ' Comments').replace('1 Comments', '1 Comment'))
ct.collect()

[('34', '1 Comment'),
 ('66', '1 Comment'),
 ('24', '4 Comments'),
 ('21', '345 Comments'),
 ('115', '8 Comments'),
 ('53', '1 Comment'),
 ('84', '44 Comments'),
 ('157', '2 Comments'),
 ('63', '2 Comments'),
 ('146', '4 Comments'),
 ('14', '21 Comment'),
 ('158', '18 Comments'),
 ('26', '5 Comments'),
 ('116', '5 Comments'),
 ('189', '2 Comments'),
 ('70', '2 Comments'),
 ('173', '20 Comments'),
 ('113', '4 Comments'),
 ('227', '20 Comments'),
 ('216', '2 Comments'),
 ('229', '3 Comments'),
 ('249', '4 Comments'),
 ('375', '13 Comments'),
 ('387', '1 Comment'),
 ('141', '4 Comments'),
 ('413', '1 Comment'),
 ('386', '9 Comments'),
 ('560', '3 Comments'),
 ('598', '2 Comments'),
 ('496', '1 Comment'),
 ('108', '23 Comments'),
 ('515', '9 Comments'),
 ('514', '3 Comments'),
 ('122', '15 Comments'),
 ('50', '1 Comment'),
 ('456', '1 Comment'),
 ('791', '8 Comments'),
 ('457', '3 Comments'),
 ('762', '7 Comments'),
 ('780', '3 Comments'),
 ('675', '2 Comments'),
 ('548', '17 Comments'),
 

In [79]:
#Calculating total score
cs = c.map(lambda x: [item for item in x[1] if item[0] in ['UserId', 'Score']])
cs = cs.filter(lambda x: len(x) > 1)
cs = cs.map(lambda x: (x[1][1].replace('"', ''), int(x[0][1].replace('"', ''))))
cs = cs.reduceByKey(lambda x,y : x + y)
cs = cs.mapValues(lambda x: ('Total Score = ' + str(x)))
cs.collect()

[('34', 'Total Score = 9'),
 ('66', 'Total Score = 3'),
 ('24', 'Total Score = 10'),
 ('21', 'Total Score = 150'),
 ('115', 'Total Score = 10'),
 ('53', 'Total Score = 0'),
 ('84', 'Total Score = 24'),
 ('157', 'Total Score = 4'),
 ('63', 'Total Score = 1'),
 ('146', 'Total Score = 2'),
 ('14', 'Total Score = 12'),
 ('158', 'Total Score = 5'),
 ('26', 'Total Score = 1'),
 ('116', 'Total Score = 3'),
 ('189', 'Total Score = 0'),
 ('70', 'Total Score = 4'),
 ('173', 'Total Score = 15'),
 ('113', 'Total Score = 6'),
 ('227', 'Total Score = 13'),
 ('216', 'Total Score = 17'),
 ('229', 'Total Score = 0'),
 ('249', 'Total Score = 1'),
 ('375', 'Total Score = 4'),
 ('387', 'Total Score = 2'),
 ('141', 'Total Score = 0'),
 ('413', 'Total Score = 0'),
 ('386', 'Total Score = 1'),
 ('560', 'Total Score = 0'),
 ('598', 'Total Score = 15'),
 ('496', 'Total Score = 1'),
 ('108', 'Total Score = 18'),
 ('515', 'Total Score = 26'),
 ('514', 'Total Score = 0'),
 ('122', 'Total Score = 6'),
 ('50', 'Tot

In [80]:
#Calculating Number of posts
p = ph.map(lambda x: [item for item in x[1] if(item[0] in ['UserId', 'PostId'])])
p = p.filter(lambda x: len(x) > 1)
p = p.map(lambda x: (x[1][1].replace('"', ''), 1))#x[0][1].replace('"', '')))
p = p.reduceByKey(lambda x, y: x + y)
p = p.mapValues(lambda x: (str(x) + ' Posts').replace('1 Posts', '1 Post'))
p.collect()

[('22', '3 Posts'),
 ('66', '6 Posts'),
 ('63', '8 Posts'),
 ('97', '186 Posts'),
 ('118', '18 Posts'),
 ('53', '10 Posts'),
 ('132', '4 Posts'),
 ('134', '17 Posts'),
 ('151', '7 Posts'),
 ('119', '3 Posts'),
 ('178', '18 Posts'),
 ('24', '11 Post'),
 ('189', '5 Posts'),
 ('199', '15 Posts'),
 ('113', '10 Posts'),
 ('34', '7 Posts'),
 ('70', '1 Post'),
 ('212', '3 Posts'),
 ('122', '69 Posts'),
 ('227', '15 Posts'),
 ('201', '286 Posts'),
 ('262', '4 Posts'),
 ('275', '3 Posts'),
 ('278', '2 Posts'),
 ('71', '4 Posts'),
 ('295', '7 Posts'),
 ('43', '2 Posts'),
 ('432', '3 Posts'),
 ('451', '1 Post'),
 ('456', '5 Posts'),
 ('454', '9 Posts'),
 ('498', '3 Posts'),
 ('514', '7 Posts'),
 ('531', '3 Posts'),
 ('655', '1 Post'),
 ('680', '1 Post'),
 ('471', '111 Post'),
 ('714', '3 Posts'),
 ('839', '3 Posts'),
 ('515', '3 Posts'),
 ('872', '3 Posts'),
 ('889', '1 Post'),
 ('322', '48 Posts'),
 ('901', '1 Post'),
 ('913', '3 Posts'),
 ('960', '9 Posts'),
 ('984', '18 Posts'),
 ('986', '6 Po

In [81]:
#Collecting User Displaynames
u = users.mapValues(lambda x: ' '.join([item[1].replace('"', '') for item in x if item[0] == 'DisplayName']) )
u.collect()

[('-1', 'Community'),
 ('1', 'Adam Lear'),
 ('2', 'Geoff Dalgas'),
 ('3', 'hichris123'),
 ('4', 'Ben Collins'),
 ('5', 'Doorknob'),
 ('6', 'gerrit'),
 ('7', 'Undo'),
 ('8', 'Jon Ericson'),
 ('9', 'rolfl'),
 ('10', 'Dave Kincaid'),
 ('11', 'Yavar'),
 ('12', 'hairboat'),
 ('14', 'Tim Goodman'),
 ('15', 'Tony Boyles'),
 ('16', 'Jeremy Banks'),
 ('17', 'Matias Valdenegro'),
 ('18', 'Gilles'),
 ('19', 'Max Ionov'),
 ('20', 'TheDoctor'),
 ('21', 'Sean Owen'),
 ('22', 'Nick Peterson'),
 ('23', 'zengr'),
 ('24', 'buruzaemon'),
 ('25', 'kabb'),
 ('26', 'Alex I'),
 ('27', 'Cole'),
 ('28', 'JorgeArtware'),
 ('29', 'Eric Wilson'),
 ('30', 'Sklivvz'),
 ('31', 'Olexandr Isayev'),
 ('32', 'Adam Barthelson'),
 ('33', 'Uri Laserson'),
 ('34', 'Vass'),
 ('35', 'Jaydles'),
 ('36', 'statsRus'),
 ('37', 'Revious'),
 ('38', 'Josh Crozier'),
 ('39', 'user1754606'),
 ('40', 'rateldajer'),
 ('41', 'Ana'),
 ('42', 'Max Makarochkin'),
 ('43', 'cynddl'),
 ('45', 'user88377'),
 ('46', 'WOPR'),
 ('47', 'ailnlv'),
 

In [82]:
#Merging ct, cs and p
a = u.leftOuterJoin(ct)
a = a.mapValues(lambda x: (x[0], str(x[1]).replace('None', 'No Comments')))
a.collect()

[('4', ('Ben Collins', 'No Comments')),
 ('10', ('Dave Kincaid', 'No Comments')),
 ('12', ('hairboat', 'No Comments')),
 ('16', ('Jeremy Banks', 'No Comments')),
 ('20', ('TheDoctor', 'No Comments')),
 ('24', ('buruzaemon', '4 Comments')),
 ('26', ('Alex I', '5 Comments')),
 ('40', ('rateldajer', 'No Comments')),
 ('50', ('Robert Cartaino', '1 Comment')),
 ('53', ('Clayton', '1 Comment')),
 ('54', ('Daniel', 'No Comments')),
 ('56', ('user3040444', 'No Comments')),
 ('57', ('twfx', 'No Comments')),
 ('60', ('NivF007', 'No Comments')),
 ('64', ('Dan C', 'No Comments')),
 ('70', ('Apollo', '2 Comments')),
 ('74', ('pkerl', 'No Comments')),
 ('77', ('Srikar Appalaraju', '6 Comments')),
 ('82', ('Damian Melniczuk', '6 Comments')),
 ('83', ('sashkello', 'No Comments')),
 ('86', ('Derek Hodgson', 'No Comments')),
 ('88', ('Igor Bobriakov', 'No Comments')),
 ('102', ('Bryan', '1 Comment')),
 ('108', ('rapaio', '23 Comments')),
 ('110', ('dejoro', 'No Comments')),
 ('111', ('Anders R. Bystrup'

In [83]:
a = a.leftOuterJoin(cs)
a = a.mapValues(lambda x: (x[0], str(x[1]).replace('None', 'Total Score = 0')))
a.collect()

[('4', (('Ben Collins', 'No Comments'), 'Total Score = 0')),
 ('16', (('Jeremy Banks', 'No Comments'), 'Total Score = 0')),
 ('20', (('TheDoctor', 'No Comments'), 'Total Score = 0')),
 ('24', (('buruzaemon', '4 Comments'), 'Total Score = 10')),
 ('40', (('rateldajer', 'No Comments'), 'Total Score = 0')),
 ('53', (('Clayton', '1 Comment'), 'Total Score = 0')),
 ('54', (('Daniel', 'No Comments'), 'Total Score = 0')),
 ('56', (('user3040444', 'No Comments'), 'Total Score = 0')),
 ('70', (('Apollo', '2 Comments'), 'Total Score = 4')),
 ('74', (('pkerl', 'No Comments'), 'Total Score = 0')),
 ('77', (('Srikar Appalaraju', '6 Comments'), 'Total Score = 1')),
 ('83', (('sashkello', 'No Comments'), 'Total Score = 0')),
 ('102', (('Bryan', '1 Comment'), 'Total Score = 0')),
 ('111', (('Anders R. Bystrup', 'No Comments'), 'Total Score = 0')),
 ('113', (('vefthym', '4 Comments'), 'Total Score = 6')),
 ('119', (('Marc Claesen', '3 Comments'), 'Total Score = 4')),
 ('128', (('Kenshin', 'No Comments'

In [84]:
a = a.leftOuterJoin(p)
a = a.mapValues(lambda x: (x[0], str(x[1]).replace('None', 'No Posts')))
a.collect()

[('40', ((('rateldajer', 'No Comments'), 'Total Score = 0'), 'No Posts')),
 ('53', ((('Clayton', '1 Comment'), 'Total Score = 0'), '10 Posts')),
 ('54', ((('Daniel', 'No Comments'), 'Total Score = 0'), 'No Posts')),
 ('202', ((('Rakib Ansary', 'No Comments'), 'Total Score = 0'), 'No Posts')),
 ('256', ((('mankuTimma', 'No Comments'), 'Total Score = 0'), 'No Posts')),
 ('285', ((('ElBaulP', 'No Comments'), 'Total Score = 0'), 'No Posts')),
 ('315', ((('user3499545', 'No Comments'), 'Total Score = 0'), 'No Posts')),
 ('328', ((('episodeyang', 'No Comments'), 'Total Score = 0'), 'No Posts')),
 ('348', ((('Justin Slade', 'No Comments'), 'Total Score = 0'), 'No Posts')),
 ('389', ((('miraculixx', '3 Comments'), 'Total Score = 0'), '6 Posts')),
 ('441', ((('Joerg Rings', 'No Comments'), 'Total Score = 0'), 'No Posts')),
 ('446', ((('Sheel Damani', 'No Comments'), 'Total Score = 0'), 'No Posts')),
 ('452', ((('Niket Kumar', 'No Comments'), 'Total Score = 0'), 'No Posts')),
 ('642',
  ((('Mika

In [85]:
a = a.values()
a.collect()

[((('rateldajer', 'No Comments'), 'Total Score = 0'), 'No Posts'),
 ((('Clayton', '1 Comment'), 'Total Score = 0'), '10 Posts'),
 ((('Daniel', 'No Comments'), 'Total Score = 0'), 'No Posts'),
 ((('Rakib Ansary', 'No Comments'), 'Total Score = 0'), 'No Posts'),
 ((('mankuTimma', 'No Comments'), 'Total Score = 0'), 'No Posts'),
 ((('ElBaulP', 'No Comments'), 'Total Score = 0'), 'No Posts'),
 ((('user3499545', 'No Comments'), 'Total Score = 0'), 'No Posts'),
 ((('episodeyang', 'No Comments'), 'Total Score = 0'), 'No Posts'),
 ((('Justin Slade', 'No Comments'), 'Total Score = 0'), 'No Posts'),
 ((('miraculixx', '3 Comments'), 'Total Score = 0'), '6 Posts'),
 ((('Joerg Rings', 'No Comments'), 'Total Score = 0'), 'No Posts'),
 ((('Sheel Damani', 'No Comments'), 'Total Score = 0'), 'No Posts'),
 ((('Niket Kumar', 'No Comments'), 'Total Score = 0'), 'No Posts'),
 ((('Mikael Vejdemo-Johansson', 'No Comments'), 'Total Score = 0'),
  'No Posts'),
 ((('citizenkong', 'No Comments'), 'Total Score = 

In [86]:
#a = a.map(lambda x: (x[0][0][0], [('Number of Comments', x[0][0][1]), ('Total Score', x[0][1]), ('Number of Posts', x[1])]))
b = a.map(lambda x: ([x[0][0][0], x[0][0][1], x[1], x[0][1]], int(x[0][1].replace('Total Score = ', ''))))
b = b.sortBy(lambda x: x[1], ascending = False)
b = b.keys()
b = b.map(lambda x: 'DisplayName: ' + '; '.join([item for item in x]))
print("Answer for fifth question:\n ")
b.collect()

Answer for fifth question:
 


['DisplayName: Emre; 1246 Comments; 440 Posts; Total Score = 684',
 'DisplayName: Neil Slater; 1199 Comments; 1180 Posts; Total Score = 544',
 'DisplayName: Spacedman; 325 Comments; 111 Post; Total Score = 177',
 'DisplayName: Vaalizaadeh; 839 Comments; 1249 Posts; Total Score = 170',
 'DisplayName: Sean Owen; 345 Comments; 1193 Posts; Total Score = 150',
 'DisplayName: Martin Thoma; 277 Comments; 648 Posts; Total Score = 132',
 'DisplayName: Aditya; 627 Comments; 221 Post; Total Score = 123',
 'DisplayName: Anony-Mousse; 463 Comments; 447 Posts; Total Score = 117',
 'DisplayName: oW_; 255 Comments; 421 Post; Total Score = 101',
 'DisplayName: Dawny33; 285 Comments; 702 Posts; Total Score = 95',
 'DisplayName: JahKnows; 350 Comments; 270 Posts; Total Score = 74',
 'DisplayName: n1k31t4; 365 Comments; 591 Post; Total Score = 73',
 'DisplayName: Aleksandr Blekh; 159 Comments; 166 Posts; Total Score = 68',
 'DisplayName: Ricardo Cruz; 132 Comments; 118 Posts; Total Score = 68',
 'DisplayN